<a href="https://colab.research.google.com/github/Deadly-Smile/Data-Science-Staff/blob/main/test_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.api.preprocessing import image_dataset_from_directory
from sklearn.model_selection import train_test_split
import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Paths for training, testing, and validation images
dataset_path = '/kaggle/input/ai-generated-images-vs-real-images/'

all_data = image_dataset_from_directory(
    dataset_path,
    image_size=(256, 256),  # Resizing to 256x256
    batch_size=32,         # Adjust batch size as needed
    shuffle=True           # Shuffle the dataset
)

# Count the total number of batches
dataset_size = all_data.cardinality().numpy()

# Calculate split sizes
train_size = int(0.7 * dataset_size)  # 70% for training
val_size = int(0.15 * dataset_size)   # 15% for validation
test_size = dataset_size - train_size - val_size  # Remaining for testing

# Split the dataset
train_data = all_data.take(train_size)
remaining_data = all_data.skip(train_size)
val_data = remaining_data.take(val_size)
test_data = remaining_data.skip(val_size)

In [ ]:
preprocessing_pipeline_train = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=1./255),
    tf.keras.layers.Resizing(height=256, width=256),
    tf.keras.layers.RandomCrop(height=256, width=256),
    tf.keras.layers.RandomRotation(factor=0.2, fill_mode="reflect", interpolation="bilinear"),
    tf.keras.layers.RandomZoom(height_factor=(0.2, 0.3), fill_mode="reflect", interpolation="bilinear"),
    tf.keras.layers.RandomBrightness(factor=0.1)
])
preprocessing_pipeline_test = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=1./255),
])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Visualize a batch
for images, labels in train_data.take(1):
    plt.figure(figsize=(10, 10))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        # Convert one-hot encoded label back to scalar
        plt.title(np.argmax(labels[i].numpy()))
        plt.axis("off")


In [ ]:
train_data = train_data.map(lambda x, y: (x, tf.one_hot(tf.cast(y, tf.int32), depth=2)))
val_data = val_data.map(lambda x, y: (x, tf.one_hot(tf.cast(y, tf.int32), depth=2)))
test_data = test_data.map(lambda x, y: (x, tf.one_hot(tf.cast(y, tf.int32), depth=2)))

AUTOTUNE = tf.data.AUTOTUNE
train_data = train_data.prefetch(buffer_size=AUTOTUNE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)
val_data = val_data.prefetch(buffer_size=AUTOTUNE)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Visualize a batch
for images, labels in train_data.take(1):
    plt.figure(figsize=(10, 10))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        # Convert one-hot encoded label back to scalar
        plt.title(np.argmax(labels[i].numpy()))
        plt.axis("off")


In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.Input(shape=(256, 256, 3)))
cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
cnn.add(tf.keras.layers.MaxPooling2D((3, 3)))
cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
cnn.add(tf.keras.layers.MaxPooling2D((3, 3)))
cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
cnn.add(tf.keras.layers.MaxPooling2D((3, 3)))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(128, activation='relu'))
cnn.add(tf.keras.layers.Dense(2, activation='softmax'))
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn.summary()

In [ ]:
cnn.fit(x = train_data, validation_data = test_data, epochs = 10)

In [ ]:
cnn.save('model.keras')